In [2]:
'''! unzip data/data136160/data.zip -d data/'''

'! unzip data/data136160/data.zip -d data/'

In [3]:
import paddle
import paddlenlp
import json
import paddle.nn as nn
import numpy as np
import jieba
import time
import os
import paddle.nn.functional as F
from functools import partial
from paddlenlp.embeddings import TokenEmbedding
from paddlenlp.data import JiebaTokenizer
from paddlenlp.data import Stack, Tuple, Pad, Dict
from paddle.io import Dataset
from tqdm import tqdm
from paddlenlp.data import JiebaTokenizer
from sklearn.metrics import f1_score
from paddlenlp import seq2vec #编码器

### 参数字典

In [37]:
paramparameters_dict={'maxlength':5,'learning_rate':5e-4,'max_epoch':40,'embedding_dim':300,'output_dim':2,\
                    'eval_step':200,'batch_size':64,'log_step':70,'save_step':2000,'dropout_p':0.3,\
                    'nhead':10,'transformer_layers':6,'lstm_hidden_size':400,'encode_dim':400,'lstm_layer':2,\
                    'CNN_num_filter':128, 'ngram_filter_sizes':(3, ),\
                    'dim_feedforward':100,'save_path':'model/','data_savepath':'data_save/','encoder_name':'transformer'}

读取数据并展示部分数据

In [ ]:
'''#数据预处理，
data_list=[]
data={}
new_data_list=[]
posi=0
nega=0
path='data_new/dev.json'
with open(path,'r',encoding='utf-8') as f:
    for line in tqdm(f.readlines()):
        d=json.loads(line)
        sentence1=d['sentence1']
        sentence2=d['sentence2']
        lnegths=(len(sentence1)+len(sentence2))/2
        new_data_list.append((lnegths,d))
        data_list.append(d)
f.close()
#序列排序
new_data_list.sort(key=lambda tup: tup[0])
#写入文件：
with open(path,'w+') as f:
    for length,d in new_data_list:
        f.write(json.dumps(d,ensure_ascii=False)+'\n')
f.close()
print("数据集大小为：",len(data_list))
test_sample=data_list[2].copy()
print(data_list[10])
print('positive sample:{:},negative sample:{:}'.format(posi,nega))'''

## 定义数据加载器（包含分次、停词、词嵌入等预处理）

#### 制作词表

In [ ]:
#加载paddle预训练的词嵌入模型,并不训练
token_embedding = TokenEmbedding(embedding_name="w2v.baidu_encyclopedia.target.word-word.dim300")

In [ ]:
#制作词表的类
class ToWordDict():
    def __init__(self,data_path,token_embeding=token_embedding,embedding_dim=300):
        self.data_path=data_path
        self.token_embeding=token_embeding
        self.tokenizer = JiebaTokenizer(vocab=token_embedding.vocab)
        self.word_dic={}
        self.word_freq_dict = dict()#记录词频
        self.word2id_dict = dict()# 每个词到id的映射关系：word2id_dict
        self.word2id_freq = dict()# 每个id出现的频率：word2id_freq
        self.id2word_dict = dict()# 每个id到词的映射关系：id2word_dict
        self.embedding_dim=embedding_dim
    
    
    def word2id_vector(self,words,mode='id'):
        if mode=='id':
            return self.tokenizer.encode(words)
        else:
            return self.token_embeding.search(words)
    
    def sentence2word(self,sentence):
        words=self.tokenizer.cut(sentence)
        return words
    
    def json2sentence(self):
        self.data=[]
        files=os.listdir(self.data_path)
        #files.remove('.ipynb_checkpoints')
        for i in files:
            with open(self.data_path+'/'+i,'r',encoding='utf-8') as f:
                for line in f.readlines():
                    d=json.loads(line)
                    self.data.append(d['sentence1'])
                    self.data.append(d['sentence2'])
            f.close()
    
    def ToWordDict(self,data=None):
        if data is not None:
            self.data=data
        for sentence in self.data:
            #切词
            words=self.sentence2word(sentence)
            #统计并制作词典
            for word in words:
                if word not in self.word_freq_dict:
                    self.word_freq_dict[word] = 0
                self.word_freq_dict[word] += 1
        #加入padding
        self.word_freq_dict['PAD'] = 0
        # 一般来说，出现频率高的高频词往往是：I，the，you这种代词，而出现频率低的词，往往是一些名词，如：nlp
        self.word_freq_dict = sorted(self.word_freq_dict.items(), key = lambda x:x[1], reverse = True)
        #构建三大词典
        # 按照频率，从高到低，开始遍历每个单词，并为这个单词构造一个独一无二的id
        for word, freq in self.word_freq_dict:
            curr_id = len(self.word2id_dict)
            self.word2id_dict[word] = curr_id
            self.word2id_freq[self.word2id_dict[word]] = freq
            self.id2word_dict[curr_id] = word
        
        # 把语料转换为id序列
    def convert_corpus_to_id(self,corpus):
        # 使用一个循环，将语料中的每个词替换成对应的id，以便于神经网络进行处理
        corpus = [self.word2id_dict[word] for word in corpus]
        return corpus
    
    def create_vocab_embedding(self):
        self.vocab_embeddings=np.zeros((len(self.word_freq_dict), self.embedding_dim))
        #print(self.word_freq_dict)
        for ind, data in enumerate(self.word_freq_dict):
            word,_=data
            embedding=self.token_embeding.search(word)
            self.vocab_embeddings[ind, :] = embedding

In [ ]:
'''twd=ToWordDict(data_path='data/data')
#josn-sentence
twd.json2sentence()
#todict
twd.ToWordDict()
#制作词表
twd.create_vocab_embedding()
#获得词表和单词-id表
vocab_embedding=twd.vocab_embeddings
word2id=twd.word2id_dict
print('[PAD]:',word2id['PAD'])
print(vocab_embedding.shape)'''

In [ ]:
'''np.save('vocab_embedding.npy',vocab_embedding)
np.save('word2id.npy',word2id)'''

In [ ]:
vocab_embedding=np.load('vocab_embedding.npy')
word2id=np.load('word2id.npy',allow_pickle=True)
word2id=word2id.item()
print(vocab_embedding.shape)
print(type(word2id))

### Embedding

#### Word2Id

In [ ]:
#将sample中的句子分词并转换为token_id序列
def convert_function(sample,token_embedding,word2id_dict):
    sentence1=sample["sentence1"]
    sentence2=sample["sentence2"]
    #使用paddleAIP进行分词，词汇为预训练词表
    tokenizer = JiebaTokenizer(vocab=token_embedding.vocab)
    words_id_1,words_id_2=[],[]
    for word in sentence1:
        if word  in word2id_dict.keys():
            words_id_1.append(word2id_dict[word])
        else:
            words_id_1.append(word2id_dict['PAD'])
    for word in sentence2:
        if word  in word2id_dict.keys():
            words_id_2.append(word2id_dict[word])
        else:
            words_id_2.append(word2id_dict['PAD'])
    sample["sentence1"]=words_id_1
    sample["sentence2"]=words_id_2
    if 'label' in sample.keys():
        sample['label']=np.array(sample['label'],dtype="float32")
    return sample
#分词、去停词、转换为id预处理
trans_func = partial(
        convert_function,
        token_embedding=token_embedding,word2id_dict=word2id)

In [ ]:
'''#trans_func示例
print(trans_func(test_sample))'''

#### embeding

In [ ]:
def word_embedding(words_id_list,vocab_embedding):
    embeddings=[vocab_embedding[i] for i in words_id_list]
    return np.array(embeddings)
embedding_func=partial(word_embedding,vocab_embedding=vocab_embedding)

### 序列转换为token_id

In [ ]:
'''original_path='data/data/train.json'
target_path='data_new/train.json'
with open(original_path,'r',encoding='utf-8') as f1:
    with open(target_path,'w+') as f2:
        for line in tqdm(f1.readlines()):
            d=json.loads(line)
            d=trans_func(d)
            label=d['label']
            d['label']=int(label)
            f2.write(json.dumps(d, ensure_ascii=False)+'\n')
    f2.close()
f1.close()'''

###  定义数据集

In [ ]:
class Dataset(paddle.io.Dataset):
    def __init__(self,data_path='data/data/',mode='train',trans_fun=None,word_embeding_func=None,if_offline_to_id=False):
        super(Dataset, self).__init__()
        self.data=[]
        self.mode=mode
        self.trans_func=trans_fun
        self.word_embedding_func=word_embeding_func
        self.if_offline_to_id=if_offline_to_id
        #读取数据
        data_path=data_path+self.mode+'.json'
        with open(data_path,'r',encoding='utf-8') as f:
            for line in f.readlines():
                d=json.loads(line)
                self.data.append(d)
        f.close()
    
    def __getitem__(self,index,cut_embedding='cut'):
        #更具具体需求返回分词与否、词嵌入与否的数据
        if self.if_offline_to_id:
            cut_embedding='no'
        if cut_embedding == 'all':
            sample=self.data[index]
            #print(sample)
            #分词去停词
            sample_words=self.trans_func(sample)
            #进行词嵌入
            sample_embedding=dict()
            sample_embedding['sentence1']=self.word_embedding_func(sample_words['sentence1'])
            sample_embedding['sentence2']=self.word_embedding_func(sample_words['sentence2'])
            if 'label' in sample.keys():
                sample_embedding['label']=sample_words['label']
            return sample_embedding
        elif cut_embedding=='cut':
            sample=self.data[index].copy()
            #分词去停词，返回句子ID
            sample_words=self.trans_func(sample)
            return sample_words
        else:#cut_embedding=='no'
            sample=self.data[index]
            return sample

    def __len__(self):
        return len(self.data)

In [ ]:
#展示部分数据
'''test_dataset=Dataset(data_path='data/data/',mode='train',trans_fun=trans_func,word_embeding_func=embedding_func)
print('原始数据：')
print(test_dataset.__getitem__(0,cut_embedding='no'))
print('分词、转换为token_id的对应数据：')
print(test_dataset.__getitem__(0,cut_embedding='cut'))
print('分词、去停词并进行词嵌入后的对应数据：')
sampletest=test_dataset.__getitem__(0,cut_embedding='all')
print("词嵌入后的维度：",sampletest['sentence1'].shape)'''

#线下计算token-id,直接返回原始数据即可
test_dataset=Dataset(data_path='data_new/',mode='train',trans_fun=trans_func,word_embeding_func=embedding_func,if_offline_to_id=True)
print('原始数据：')
print(test_dataset.__getitem__(0,cut_embedding='no'))

# 保存数据！！！！！！

### 定义数据集加载器

In [ ]:
#构建数据加载器
def create_dataloader(dataset, mode='train',batch_size=1,batchify_fn=None):
    shuffle = True if mode == 'train' else False
    if mode == 'train':
        batch_sampler = paddle.io.DistributedBatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        batch_sampler = paddle.io.BatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    return paddle.io.DataLoader(dataset=dataset,batch_sampler=batch_sampler,collate_fn=batchify_fn,return_list=True)

#整合Batch的数据
#需要根据是否使用Bert进行调整：非Bert返回token-id，Bert则需返回token-id\token_type等为字典。
def collate_func(batch_data,if_bert=False,max_length=15):
    batch_size = len(batch_data)
    # 如果batch_size为0，则返回一个空字典
    if batch_size == 0:
        return {}
    sentence1_list,sentence2_list,label_list=[],[],[]
    input_ids,token_type_ids=[],[]
    #判断是否是预测数据
    instance1=batch_data[1]
    is_test=1
    if 'label' in instance1.keys():
        is_test=0
    for instance in batch_data:
        if if_bert:
            token_type_id=instance['token_type_ids']
            input_id=instance['input_ids']
            input_ids.append(paddle.to_tensor(input_id, dtype="int64"))
            token_type_ids.append(paddle.to_tensor(token_type_id, dtype="int64"))
        else:
            sentence1=instance["sentence1"]
            sentence2=instance["sentence2"]
            #由于是token-id因此返回的是int64形
            sentence1_list.append(paddle.to_tensor(sentence1, dtype="int64"))
            sentence2_list.append(paddle.to_tensor(sentence2, dtype="int64"))
        if is_test==0:
            label=instance['label']
        if is_test==0:
            label_list.append(label)
    # 对一个batch内的数据，进行padding,padding的值为词表中的[pad]的Index:635964
    if if_bert:
        if is_test==0:
            return {'input_ids':Pad(axis=0, pad_val=tokenizer.pad_token_id,pad_right=False)(input_ids),  # input_ids
                'token_type_ids':Pad(axis=0, pad_val=tokenizer.pad_token_type_id,pad_right=False)(token_type_ids),
                "labels":Stack(dtype="int64")(label_list),
                }
        else:
            return {'input_ids':Pad(axis=0, pad_val=tokenizer.pad_token_id,pad_right=False)(input_ids),
                'token_type_ids':Pad(axis=0, pad_val=tokenizer.pad_token_type_id,pad_right=False)(token_type_ids),
                }
    else:
        if is_test==0:
            return {"sentence1s": Pad(pad_val=10218, axis=0,pad_right=False)(sentence1_list),
                "sentence2s": Pad(pad_val=10218, axis=0,pad_right=False)(sentence2_list),
                "labels": Stack(dtype="int64")(label_list),
                }
        else:
            return {"sentence1s": Pad(pad_val=10218, axis=0,pad_right=False)(sentence1_list),
                "sentence2s": Pad(pad_val=10218, axis=0,pad_right=False)(sentence2_list),
                }

In [ ]:
#测试：
#构建数据集
batch_size=paramparameters_dict['batch_size']
test_data_loader = create_dataloader(test_dataset,mode='train',batch_size=batch_size,batchify_fn=collate_func)
print(test_dataset.__len__()//batch_size)


### 编码器构建

In [ ]:
#LSTM
#Lstm使用最后一个输出编码作为整体句子的编码，对编码的句子的长度没有要求
lstm_encoder=nn.LSTM(input_size=paramparameters_dict['embedding_dim'],\
                hidden_size=paramparameters_dict['encode_dim'],num_layers=2,dropout=paramparameters_dict['dropout_p'])

#使用集成的LSTMEncoder
lstm_encoder_api=seq2vec.LSTMEncoder(input_size=paramparameters_dict['embedding_dim'],hidden_size=paramparameters_dict['encode_dim'],\
                num_layers=paramparameters_dict['lstm_layer'],direction='bidirect')

#CNN
#使用集成的CNNEncoder
cnn_encoder_api=seq2vec.CNNEncoder(emb_dim=paramparameters_dict['embedding_dim'],num_filter=paramparameters_dict['CNN_num_filter'],\
                ngram_filter_sizes=paramparameters_dict['ngram_filter_sizes'])

#RNN集成API
rnn_encoder_api=seq2vec.RNNEncoder(input_size=paramparameters_dict['embedding_dim'],\
            hidden_size=paramparameters_dict['encode_dim'],\
            num_layers=paramparameters_dict['lstm_layer'],)


'''from paddlenlp.transformers import BertModel,BertTokenizer

#加载预训练的Bert模型
tokenizer_bert = BertTokenizer.from_pretrained('bert-wwm-chinese')
bert_encode = BertModel.from_pretrained('bert-wwm-chinese')
#Ernie
tokenizer_ernie = paddlenlp.transformers.ErnieGramTokenizer.from_pretrained('ernie-gram-zh')
Ernie_encode = paddlenlp.transformers.ErnieGramModel.from_pretrained('ernie-gram-zh')

tokenizer=tokenizer_bert

#对于Bert需要特有的转换方式等预处理方式,输入sample字典，输出是每个句子对应的input_id\token_type_id的字典
def for_Bert_transfunc(sample,tokenizer,max_seq_length=512):
    query=sample['sentence1']
    title=sample['sentence2']
    encoded_inputs = tokenizer(
        text=query, text_pair=title, max_seq_len=max_seq_length)
    if 'label' in sample.keys():
        encoded_inputs['label']=np.array(sample['label'],dtype="float32")
    return encoded_inputs

bert_transfunc=partial(for_Bert_transfunc,tokenizer=tokenizer,max_seq_length=512)'''

In [34]:
#transformer编码器
from paddlenlp.transformers import WordEmbedding,PositionalEmbedding
class Transformer_encoder(nn.Layer):
    def __init__(self,vocab_size,max_length,emb_dim,encode_dim,\
        trans_layer_num,trans_head_num,hidden_dim,normalize_before=False,if_pre_embeding=False,token_embedding=vocab_embedding ):
        super(Transformer_encoder,self).__init__()
        #输入：词嵌入大小，最大长度，词嵌入维度，句子编码维度，transformer层数，\
        # 多头机制头数目，FNN隐藏层大小,是否在子层输入前层归一化
        #定义wordembedding和positionembeding，作为transformer的输入
        
        self.if_pre_embeding=if_pre_embeding
        #选择是否使用预训练的词嵌入
        if self.if_pre_embeding:
            print('使用预训练的词向量')
            pretrained_attr = paddle.ParamAttr(
                                   initializer=paddle.nn.initializer.Assign(token_embedding),
                                   trainable=False)
            self.wordembedding=nn.Embedding(num_embeddings=token_embedding.shape[0],
                                      embedding_dim=token_embedding.shape[1],
                                      weight_attr=pretrained_attr)
        else:
            print('使用可学习的词向量')
            self.wordembedding=WordEmbedding(vocab_size=vocab_size,emb_dim=emb_dim,bos_id=10218)
        self.positionembedding=PositionalEmbedding(emb_dim=emb_dim,max_length=max_length)
        #定义编码层
        self.encoderlayer=nn.TransformerEncoderLayer(d_model=emb_dim,nhead=trans_head_num,\
            dim_feedforward=hidden_dim,normalize_before=normalize_before)
        #定义编码器：
        self.tansformer_encoder=nn.TransformerEncoder(self.encoderlayer,num_layers=trans_layer_num)
        #定义线性层，将transformer的输出映射到编码输出
        self.liner=nn.Linear(in_features=emb_dim,out_features=encode_dim)
    
    #将transformer encoder的每一个输出加和，作为句子表征
    def concat_trans_out(self,code):
        encode=paddle.zeros((code.shape[0],code.shape[2]))
        for i in range(code.shape[1]):
            encode=code[:,i,:]+encode
        return encode

    def avrage2out(self,code):
        encode=paddle.zeros((code.shape[0],code.shape[2]))
        for i in range(code.shape[1]):
            encode=code[:,i,:]+encode
        return encode/code.shape[1]

    def maxpooling2out(self,code):
        encode=paddle.max(code,axis=1)
        return encode
    
    def forward(self,token_ids):
        #[batch_size,sequence_length]
        batch_size,sequence_length=token_ids.shape
        #首先得到对饮的位置id
        pos=paddle.tile(paddle.arange(start=0, end=sequence_length), repeat_times=[batch_size, 1])
        #进行位置编码
        pos_emb = self.positionembedding(pos)
        #进行词嵌入
        src_emb = self.wordembedding(token_ids)
        #加和作为transformer输入
        trans_input=pos_emb+src_emb
        code=self.tansformer_encoder(trans_input)
        #print(code)
        code=self.liner(code)
        #print(code.shape)
        #得到句子表征
        out=self.maxpooling2out(code)
        return out

transformer_encoder=Transformer_encoder(vocab_size=vocab_embedding.shape[0],max_length=256,emb_dim=paramparameters_dict['embedding_dim'],\
        encode_dim=paramparameters_dict['encode_dim'],trans_layer_num=paramparameters_dict['transformer_layers'],\
        trans_head_num=paramparameters_dict['nhead'],hidden_dim=paramparameters_dict['dim_feedforward'],normalize_before=True,if_pre_embeding=True)

In [ ]:
'''test_sample={'sentence1': '支付宝系统点我的里面没有花呗这一项', 'sentence2': '我下载支付宝怎么没有花呗的', 'label': '1'}
print(test_sample)
max_seq_length=512
encoded_inputs = bert_transfunc(test_sample)
print(encoded_inputs)'''

In [ ]:
'''#通过Bert转换后的示例
test_sample={'sentence1': '蚂蚁花呗说我违约一次', 'sentence2': '蚂蚁花呗违约行为是什么', 'label': '0'}
print('原始数据：')
print(test_sample)
input_sample=bert_transfunc(test_sample)
print('转换后数据')
input_ids=paddle.to_tensor([input_sample['input_ids']],dtype='int64')
token_type_ids=paddle.to_tensor([input_sample['token_type_ids']],dtype='int64')
print(input_sample)
print('cls向量')
(sequence_output,cls)=Ernie_encode(input_ids=input_ids,token_type_ids=token_type_ids)
print(cls.shape)'''

### 分类器构建，组网

In [ ]:
class Classifer(nn.Layer):
    def __init__(self,encoder=None,embedding_dim=paramparameters_dict['embedding_dim'],\
                encoder_dim=paramparameters_dict['encode_dim'],\
                class_dim=paramparameters_dict['output_dim'],is_embedding=False,\
                token_embedding=vocab_embedding,is_pre_embedding=False,mode='LSTM',init_scale=0.1,vocab_size=vocab_embedding.shape[0],):
        #参数分别的意义：编码器、词嵌入维度、编码维度、分类数目、是否进行词嵌入、词嵌入的词表、是否使用预训练词嵌入、编码器名称、词嵌入层初始化、词表大小
        super(Classifer,self).__init__()
        print('使用的编码器是：'+mode)
        self.is_pre_embedding=is_pre_embedding
        self.is_embedding=is_embedding
        self.embedding_dim=embedding_dim
        self.encoder_dim=encoder_dim
        self.mode=mode
        self.encoder=encoder
        if self.is_pre_embedding and self.is_embedding:
            #构建预训练的词嵌入层
            print('使用预训练的词嵌入')
            pretrained_attr = paddle.ParamAttr(
                                   initializer=paddle.nn.initializer.Assign(token_embedding),
                                   trainable=False)
            self.embedding_layer=nn.Embedding(num_embeddings=token_embedding.shape[0],
                                      embedding_dim=token_embedding.shape[1],
                                      weight_attr=pretrained_attr)
        elif self.is_embedding:
            #使用非预训练的词嵌入层
            print('使用非预训练的词嵌入，训练词嵌入层')
            self.embedding_layer=nn.Embedding(num_embeddings=vocab_size, embedding_dim=self.embedding_dim, sparse=False, 
                                    weight_attr=paddle.ParamAttr(initializer=nn.initializer.Uniform(low=-init_scale, high=init_scale)))
                                    
        if self.mode!='Bert' and self.mode!="Ernie":
            self.fnn=nn.Sequential(nn.Linear(in_features=2*self.encoder_dim,out_features=400),nn.ReLU(),\
                    nn.Linear(in_features=400,out_features=class_dim))
            print('使用普通的Embedding层')

        else:
            self.fnn=nn.Sequential(nn.Linear(in_features=self.encoder.config["hidden_size"],out_features=400),nn.ReLU(),\
                    nn.Linear(in_features=400,out_features=class_dim))
            print('使用Transformer-based model')
        
    def forward(self,input_data):
        first_sentence_id,second_sentence_id=input_data[0],input_data[1]
        #如果不是Bert,那么直接就是输入词向量或者token-id
        #对于Bert编码器，两个sentence_id对应的是各自句子的Bert：input_id和token_type_id，需要进一步处理转换为Bert的输入，且使用Bert时，不需要embedding。
        if self.is_embedding:
            #进行词嵌入
            first_sentence=self.embedding_layer(first_sentence_id)
            second_sentence=self.embedding_layer(second_sentence_id)
        else:
            first_sentence=first_sentence_id
            second_sentence=second_sentence_id
        #计算编码
        if self.mode=='Bert' or self.mode=='Ernie':
            (sequence_output,cls)=self.encoder(input_ids=first_sentence,\
                                    token_type_ids=second_sentence)
            #得到的CLS就可以用于文本分类
            input_v=cls
            out=self.fnn(input_v)
            return out
        
        elif self.mode=='Transformer':
            code1=self.encoder(first_sentence)
            code2=self.encoder(second_sentence)
            #合并向量：
            input_v=paddle.concat(x=[code1,code2], axis=-1)
            out=self.fnn(input_v)
            return out
        elif  self.mode=='LSTM_API':
            seq_len1=paddle.to_tensor([first_sentence.shape[1] for i in range(first_sentence.shape[0])])
            seq_len2=paddle.to_tensor([second_sentence.shape[1] for i in range(second_sentence.shape[0])])
            code1=self.encoder(first_sentence,seq_len1)
            code2=self.encoder(second_sentence,seq_len2)
            #合并向量：
            input_v=paddle.concat(x=[code1,code2], axis=-1)
            out=self.fnn(input_v)
            return out
        elif self.mode=='CNN':
            code1=self.encoder(first_sentence)
            code2=self.encoder(second_sentence)
            #合并向量：
            input_v=paddle.concat(x=[code1,code2], axis=-1)
            out=self.fnn(input_v)
            return out
        elif self.mode=='LSTM':
            codes1,(h,c)=self.encoder(first_sentence)
            codes2,(h,c)=self.encoder(second_sentence)
            #选择句子表征
            code1=self.sumtosentencevecter(codes1)
            code2=self.sumtosentencevecter(codes2)
             #合并向量：
            input_v=paddle.concat(x=[code1,code2], axis=-1)
            out=self.fnn(input_v)
            return out

### 预测函数

In [ ]:
def alltest(model,data_loader_test):
    # 测试函数，需要完成的任务有：根据测试数据集中的数据，逐个对其进行预测，生成预测值。
    with open('predict_labels_1120192305.txt','w+',encoding='utf-8') as f:
        model.eval()
        mode=model.mode
        for sample in tqdm(data_loader_test):
            if mode=="Bert" or mode=='Ernie':
                sentence1=sample['input_ids']
                sentence2=sample['token_type_ids']
            else:
                sentence1=sample['sentence1s']
                sentence2=sample['sentence2s']
            #预测
            input_data=[sentence1,sentence2]
            pres=model(input_data)
            pres=np.argmax(pres.numpy(),axis=-1)
            for pre in pres:
                f.write(str(pre)+'\n')
    f.close()

### 验证函数

In [ ]:
def evaluate(model, criterion, metric, data_loader):
    model.eval()
    metric.reset()
    losses = []
    acces=[]
    f1score=[]
    mode=model.mode
    for sample in data_loader:
        if mode=="Bert" or mode=='Ernie':
            sentence1=sample['input_ids']
            sentence2=sample['token_type_ids']
        else:
            sentence1=sample['sentence1s']
            sentence2=sample['sentence2s']
        labels=sample['labels']
        input_data=[sentence1,sentence2]
        logits = model(input_data)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        logits = F.softmax(logits, axis=1)
        correct = metric.compute(logits, labels)
        #f1分数
        y_pred=np.argmax(logits.numpy(),axis=-1)
        y_ture=labels.numpy()
        f1=f1_score(y_pred=y_pred,y_true=y_ture)
        f1score.append(f1)
        metric.update(correct)
        accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f,f1:%.5f" % (np.mean(losses),accu,np.mean(f1score)))
    model.train()
    metric.reset()
    #返回验证的损失和准确率便于数据展示
    return (np.mean(losses), accu,np.mean(f1score))

In [ ]:
def do_train( model, data_loader,  vali_data_loader, criterion,  optimizer,metric , scheduler=None  ):
    #paddle.set_device('gpu:0')
    model.train()
    global_step = 0

    tic_train = time.time()
    num_train_epochs=paramparameters_dict['max_epoch']
    log_steps=paramparameters_dict['log_step']
    #log_steps=1
    mode=model.mode#使用的编码器
    eval_step=paramparameters_dict['eval_step']
    #eval_step=20
    save_step=paramparameters_dict['save_step']
    train_loss,train_acc,train_f1,train_iters=[],[],[],[]
    eval_loss,eval_acc,eval_iters,eval_f1=[],[],[],[]
    for epoch in range(num_train_epochs):
        for step,sample in enumerate(data_loader):
            if mode=="Bert" or mode=='Ernie':
                sentence1=sample['input_ids']
                #print('inputid:',sentence1.shape)
                sentence2=sample['token_type_ids']
                #print('tokrn_type',sentence2)
            else:
                sentence1=sample['sentence1s']
                sentence2=sample['sentence2s']
            truelabels=sample['labels']
            #print(truelabels.shape)
            #print(type(sentence1))
            input_data=[sentence1,sentence2]
            outputs = model(input_data)
            #print(outputs.shape)
            #计算损失
            loss = criterion(outputs, truelabels)
            #print(loss)
            outputs = F.softmax(outputs, axis=1)
            correct = metric.compute(outputs, truelabels)
            metric.update(correct)
            acc = metric.accumulate()
            #f1分数
            y_pred=np.argmax(outputs.numpy(),axis=-1)
            y_ture=truelabels.numpy()
            f1=f1_score(y_pred=y_pred,y_true=y_ture)
            #反向传播
            loss.backward()
            global_step += 1
            # 每间隔 log_steps 输出训练指标
            if global_step % log_steps == 0:
                print("global step %d, epoch: %d, batch: %d, loss: %.5f, accuracy: %.5f,F1-score:%.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,f1,
                    log_steps / (time.time() - tic_train)))
                train_iters.append(global_step)
                train_acc.append(acc)
                train_f1.append(f1)
                train_loss.append(loss.numpy())
            if global_step%eval_step==0 :
                evalloss,evalacc,evalf1=evaluate(model, criterion, metric, vali_data_loader)
                eval_acc.append(evalacc)
                eval_loss.append(evalloss)
                eval_f1.append(evalf1)
                eval_iters.append(global_step)
            #优化器迭代一步
            optimizer.step()
            optimizer.clear_grad()
            if scheduler:
                scheduler.step()
            '''if global_step % save_step == 0:
                save_path=paramparameters_dict['save_path']+paramparameters_dict['encoder_name']+str(global_step)+'.pdparams'
                paddle.save(model.state_dict(),save_path)'''
        metric.reset()
    save_path=paramparameters_dict['save_path']+paramparameters_dict['encoder_name']+'fc_finall.pdparams'
    paddle.save(model.state_dict(),save_path)
    return (train_loss,train_acc,train_f1,train_iters),(eval_loss,eval_acc,eval_f1,eval_iters)

### 加载数据集

In [ ]:
#构建数据集
batch_size=paramparameters_dict['batch_size']
'''#bert模型
trans_fun=bert_transfunc
collate_fun=partial(collate_func,if_bert=True)
train_ds=Dataset(data_path='data/data/',mode='train',trans_fun=trans_fun,word_embeding_func=embedding_func,if_offline_to_id=False)
eval_ds=Dataset(data_path='data/data/',mode='dev',trans_fun=trans_fun,word_embeding_func=embedding_func,if_offline_to_id=False)'''
#lstm\transformer模型
trans_fun=trans_func
collate_fun=collate_func
train_ds=Dataset(data_path='data_new/',mode='train',trans_fun=trans_fun,word_embeding_func=embedding_func,if_offline_to_id=True)
eval_ds=Dataset(data_path='data_new/',mode='dev',trans_fun=trans_fun,word_embeding_func=embedding_func,if_offline_to_id=True)
#test_ds=Dataset(data_path='data/data/',mode='test',trans_fun=trans_fun,word_embeding_func=embedding_func)
print('训练集大小：',train_ds.__len__())
print('验证集大小：',eval_ds.__len__())
#print('测试集大小：',test_ds.__len__())
train_data_loader = create_dataloader(train_ds,mode='train',batch_size=batch_size,batchify_fn=collate_fun)
eval_data_loader = create_dataloader(eval_ds,mode='dev',batch_size=batch_size,batchify_fn=collate_fun)
#test_data_loader = create_dataloader(test_ds,mode='test',batch_size=batch_size,batchify_fn=collate_fun)
print('Batch_size:',batch_size)
print('训练一轮步数：',train_ds.__len__()//batch_size)

### 定义模型

In [35]:
#paddle.set_device('gpu:0')
#bert model
#Enire model
'''model=Classifer(encoder=bert_encode,is_embedding=False,mode='Bert')'''
#transformer
model=Classifer(encoder=transformer_encoder,is_embedding=False,mode='Transformer')
#lstm
'''model=Classifer(encoder=latm_encoder_api,is_embedding=True,is_pre_embedding=True,mode='transformer')'''
#打印网络查看网络结构
params_info = paddle.summary(model, (2, 64,5),dtypes='int64')
print(params_info)

#### 定义优化器等

In [38]:
#定义损失函数：交叉熵损失
criterion=nn.loss.CrossEntropyLoss()
#定义优化器
'''from paddlenlp.transformers import LinearDecayWithWarmup
num_training_steps = len(train_data_loader) * paramparameters_dict['max_epoch']
lr_scheduler = LinearDecayWithWarmup(paramparameters_dict['learning_rate'], num_training_steps, 0.0)
decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义 Optimizer
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=0.0,
    apply_decay_param_fun=lambda x: x in decay_params)'''
#lstm优化器
optimizer = paddle.optimizer.AdamW(learning_rate=paramparameters_dict['learning_rate'],parameters=model.parameters())
print(paramparameters_dict['learning_rate'])
#定义评估标准：准且率、二分了可以使用F1分数
acc_metric=paddle.metric.Accuracy()

### 训练模型

In [39]:
#训练验证
start=time.perf_counter()
paddle.set_device('gpu:0')
#train_data,eval_data=do_train(model,train_data_loader,eval_data_loader,criterion,optimizer,acc_metric,lr_scheduler)
train_data,eval_data=do_train(model,train_data_loader,eval_data_loader,criterion,optimizer,acc_metric)
end=time.perf_counter()
use_time=end-start
print('训练使用了：{:.5f}'.format(use_time))
#测试
#alltest(model,test_data_loader)

In [41]:
'''np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'train_loss.npy',train_data[0])
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'trainb _acc.npy',train_data[1])
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'train_f1.npy',train_data[2]) 
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'train_iter.npy',train_data[3])

np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'eval_loss.npy',eval_data[0])
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'eval_acc.npy',eval_data[1])
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'eval_f1.npy',eval_data[2])
np.save(paramparameters_dict['data_savepath']+paramparameters_dict['encoder_name']+'eval_iter.npy',eval_data[3])'''

In [40]:
print('训练集最佳准确率：{:.4f}'.format(np.max(train_data[1])))
print('验证集最佳准确率：{:.4f}'.format(np.max(eval_data[1])))
print('训练集最佳F1分数：{:.4f}'.format(np.max(train_data[2])))
print('验证集最佳F1分数：{:.4f}'.format(np.max(eval_data[2])))

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(train_data[3],train_data[0],label='train_loss')
plt.plot(train_data[3],train_data[1],label='train_acc')
plt.plot(train_data[3],train_data[2],label='train_f1')
plt.legend()
plt.title('encode is '+paramparameters_dict['encoder_name']+":train_process")
plt.ylabel('accuracy/loss/f1_score')
plt.xlabel('step')
plt.ylim(0,1)
plt.grid()
plt.show()

In [ ]:
plt.plot(eval_data[3],eval_data[0],label='eval_loss')
plt.plot(eval_data[3],eval_data[1],label='eval_acc')
plt.plot(eval_data[3],eval_data[2],label='eval_f1score')
plt.legend()
plt.title('encode is '+paramparameters_dict['encoder_name']+":eval_process")
plt.ylabel('accuracy/loss/f1_score')
plt.xlabel('step')
plt.ylim(0,1)
plt.grid()
plt.show()

In [ ]:
plt.plot(train_data[3],train_data[0],label='train_loss')
plt.plot(eval_data[3],eval_data[0],label='eval_loss')
plt.plot(train_data[3],train_data[1],label='train_acc')
plt.plot(eval_data[3],eval_data[1],label='eval_acc')
plt.plot(train_data[3],train_data[2],label='train_f1')
plt.plot(eval_data[3],eval_data[2],label='eval_f1')
plt.legend()
plt.title('encode is '+paramparameters_dict['encoder_name']+":all_process")
plt.ylabel('accuracy/loss/f1')
plt.xlabel('step')
plt.ylim(0,1)
plt.grid()

### 样本预测

In [ ]:
model.eval()
label_dict={0:'不相似',1:'相似'}
test_example={"sentence1": "本月花呗为什么不能分期", "sentence2": "花呗分期为什么不能红包付款"}
ture_label='不相似'
inputs=trans_fun(test_example)
#print(inputs)
inputs_=[paddle.to_tensor([inputs['sentence1']]),paddle.to_tensor([inputs['sentence2']])]
pre=model(inputs_)
pre=F.softmax(pre,axis=1)
pre_label=np.argmax(pre.numpy()[0])
print('句子1为：'+"本月花呗为什么不能分期")
print('句子2为：'+"花呗分期为什么不能红包付款")
print('两个句子实际'+ture_label)
print('-----------------------------------------------')
print('编码器为:'+paramparameters_dict['encoder_name'])
print('两个类别概率为:',pre.numpy()[0])
print('预测为：'+label_dict[pre_label])